In [45]:
import numpy as np 
import pandas as pd

from geopy.geocoders import Nominatim 
import geocoder 

import requests, json 
import wikipedia as wp

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

print("Libraries imported.")

Libraries imported.


In [87]:
#Import just the table from the site. I checked for the table name from the wiki page source.
table_url = wp.page("List_of_communities_in_Dubai").html().encode("UTF-8")

#determin the index of the table by first row
dubai_table = pd.read_html(table_url, header = [0])[0]
dubai_table = dubai_table.drop(dubai_table.columns[[2, 3, 4, 5]], axis=1)
dubai_table.rename(columns={'Community (English)':'communities'},inplace=True)
dubai_table.rename(columns={'Community Number':'co-nr'},inplace=True)

# Droping 'Not assigned' rows
#dubai_table = dubai_table[dubai_table.Neighbourhood != 'Not assigned']

dubai_table.head()


,co-nr,communities
0,126.0,Abu Hail
1,711.0,Al Awir First
2,721.0,Al Awir Second
3,333.0,Al Bada
4,122.0,Al Baraha


In [30]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Dubai, UAE'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coordinates = [ get_latlng(neighborhood) for neighborhood in dubai_table["communities"].tolist() ]
coordinates

[[25.28308000000004, 55.33435000000003],
 [25.186050000000023, 55.54108000000008],
 [25.167920000000038, 55.543310000000076],
 [25.21861000000007, 55.26406000000003],
 [25.282800000000066, 55.31678000000005],
 [25.11483000000004, 55.19136000000003],
 [25.107230000000072, 55.20485000000008],
 [25.08958000000007, 55.23424000000006],
 [25.077390000000037, 55.24267000000003],
 [25.062290000000075, 55.23995000000008],
 [25.093420000000037, 55.19044000000008],
 [25.269250000000056, 55.29944000000006],
 [25.272170000000074, 55.30157000000003],
 [25.243370000000027, 55.352670000000046],
 [25.269510000000025, 55.30884000000003],
 [25.25696000000005, 55.30246000000005],
 [25.29871000000003, 55.33546000000007],
 [25.237130000000036, 55.27707000000004],
 [25.220540000000028, 55.34166000000005],
 [25.233420000000024, 55.29001000000005],
 [25.245290000000068, 55.30364000000003],
 [25.27177000000006, 55.33762000000007],
 [25.24282000000005, 55.48440000000005],
 [25.22784000000007, 55.522320000000036]

In [49]:
dubai_table_coordinates = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
dubai_table['Latitude'] = dubai_table_coordinates['Latitude']
dubai_table['Longitude'] = dubai_table_coordinates['Longitude']


In [83]:
dubai_table.to_csv(r'dubai_coord.csv', index = False, header=True)

In [66]:
dubai_latitude = 25.0750095
dubai_longitude = 55.18876088183319

In [75]:
map_dubai = folium.Map(location=[dubai_latitude, dubai_longitude], zoom_start=10)

# add markers to map
for lat, lng, communities in zip(dubai_table['Latitude'], dubai_table['Longitude'], dubai_table['communities']):
    label = '{}'.format(communities)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dubai)  

map_dubai